In [1]:
#refreence:
#dropout:https://blog.csdn.net/stdcoutzyx/article/details/49022443

from __future__ import division,print_function,absolute_import
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("Mnist",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting Mnist\t10k-images-idx3-ubyte.gz
Extracting Mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate=0.1
mini_batch=128
num_steps=500
display_step=10

n_inputs=784
n_classes=10
dropout=0.75
#probability to keep units

X=tf.placeholder(tf.float32,[None,n_inputs])
Y=tf.placeholder(tf.float32,[None,n_classes])
keep_prob=tf.placeholder(tf.float32)#dropout keep probability



In [3]:
#create some wrappers for simplifacation
def conv2d(x,W,b,strides=1):
    x=tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    y=tf.nn.bias_add(x,b)
    return tf.nn.relu(y)

def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

#create model
def conv_net(x,weights,biases,dropout):
    x=tf.reshape(x,shape=[-1,28,28,1])
    #？？？？-1作为batch_size意思
    #mnist原输入是一维的，重新塑形，转成图片的格式[高度 x 宽度 x 色彩通道]
    #tensor input become 4-D[batch size, height,width,channel]
    conv1=conv2d(x,weights['wc1'],biases['bc1'])
    conv1=maxpool2d(conv1,k=2)
    #？？？k=2意思，卷积核为2，但是为真么放在最大池化的位置
    
    conv2=conv2d(conv1,weights['wc2'],biases['bc2'])
    conv2=maxpool2d(conv2,k=2)
    
    #fully connected layer
    #reshape conv2 output to fit fully connected layer input
    fc1=tf.reshape(conv2,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1=tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1=tf.nn.relu(fc1)
    fc1=tf.nn.dropout(fc1,dropout)
    
    out=tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    return out

In [4]:
weights={
    #5*5卷积
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
    #全连接
    'out':tf.Variable(tf.random_normal([1024,n_classes]))
}
biases={
    'bc1':tf.Variable(tf.random_normal([32])),
     'bc2':tf.Variable(tf.random_normal([64])),
     'bd1':tf.Variable(tf.random_normal([1024])),
     'out':tf.Variable(tf.random_normal([n_classes]))
}

logits=conv_net(X,weights,biases,keep_prob)
prediction=tf.nn.softmax(logits)

loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits,labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op=optimizer.minimize(loss_op)

correct_pred=tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))
init=tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
with tf.Session() as sess:
    print("In!!!")
    sess.run(init)
    for step in range(1,num_steps+1):
        batch_x,batch_y=mnist.train.next_batch(mini_batch)
        sess.run(train_op,feed_dict={X:batch_x,Y:batch_y,keep_prob:dropout})
    if step % display_step==0 or step==1:
        loss,acc=sess.run([loss_op,accuracy],feed_dict={X:batch_x,
                                                        Y:batch_y,
                                                        keep_prob:1.0})
        print("Step: "+str(step)+" minibatch loss= "+"{:.4f}".format(loss)+
              " Training Accuracy= "+"{:.3f}".format(acc))
    print("Optimization finished!")
    
    print("Testing Accuracy:",sess.run(accuracy,fedd_dict={X:mnist.test.images[:256],
                                                          Y:mnist.test.labels[:256],
                                                          keep_prob:1.0}))